### RAG Playground: Llama 3 8B Instruct + Reddit-style Few-shot from JSON

Goal: retrieve example Reddit posts from `./data/post-sample.json` and condition Llama 3 8B Instruct to generate one stylistically similar post.

Prompt used: "generate a reddit post that the user is likely to enjoy"


In [16]:
%pip install -qU transformers==4.55.2


Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install -qU transformers==4.55.2 sentence-transformers faiss-cpu datasets einops peft accelerate bitsandbytes jinja2>=3.1.0


Note: you may need to restart the kernel to use updated packages.


In [18]:
# Load and clean dataset; assemble corpus strings
from typing import List, Dict
import json, re, os

DATA_PATH = "./data/post-sample.json"

assert os.path.exists(DATA_PATH), f"Missing {DATA_PATH}"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw_posts: List[Dict] = json.load(f)

BOILERPLATE_PATTERNS = [
    r"^\s*View\s+More\s+Posts\s*$",
    r"^\s*View\s+Post\s*$",
    r"^\s*Help\??\s*$",
    r"^\s*Edit:\s*.*$",
]
boilerplate_regexes = [re.compile(p, flags=re.IGNORECASE) for p in BOILERPLATE_PATTERNS]

def clean_text(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r"\r\n?", "\n", text)
    text = re.sub(r"\s+", " ", text).strip()
    lines = [ln.strip() for ln in text.split("\n")]
    kept = []
    for ln in lines:
        if any(rx.match(ln) for rx in boilerplate_regexes):
            continue
        kept.append(ln)
    return "\n".join(kept).strip()

corpus: List[str] = []
meta: List[Dict] = []
for p in raw_posts:
    title = clean_text(p.get("title", ""))
    self_text = clean_text(p.get("self_text", ""))
    subreddit = clean_text(p.get("subreddit", ""))
    subreddit = re.sub(r"\s*(/)?r/", "r/", subreddit)
    doc = f"title: {title}\nself_text: {self_text}\nsubreddit: {subreddit}"
    corpus.append(doc)
    meta.append({"title": title, "subreddit": subreddit})

len(corpus), corpus[0][:200]


(250,
 'title: 3070ti, 6900xt or wait for new cards?\nself_text: Backstory: I ordered a $500 34" qhd 144hz monitor off best buy and they accidentally shipped me 2 of them so now I\'m going to return one lol. I ')

In [19]:
# Minimal embedding + retrieval without sentence-transformers/faiss
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

_emb_tok = AutoTokenizer.from_pretrained(EMBED_MODEL, use_fast=True)
_emb_model = AutoModel.from_pretrained(EMBED_MODEL)
_emb_model.eval()
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_emb_model.to(_device)

@torch.no_grad()
def _encode_texts(texts):
    toks = _emb_tok(texts, padding=True, truncation=True, max_length=256, return_tensors="pt").to(_device)
    out = _emb_model(**toks)
    token_embs = out.last_hidden_state
    mask = toks["attention_mask"].unsqueeze(-1)
    pooled = (token_embs * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
    pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
    return pooled.cpu().numpy()

embeddings = _encode_texts(corpus)

def retrieve_idxs(query: str, k: int = 6):
    q = _encode_texts([query])[0]
    scores = embeddings @ q
    k = min(k, len(scores))
    topk = np.argpartition(-scores, kth=k-1)[:k]
    topk = topk[np.argsort(-scores[topk])]
    return topk, scores[topk]

# Override retrieval to use new index
from typing import List

def retrieve_examples(query: str, k: int = 6) -> List[str]:
    idxs, _ = retrieve_idxs(query, k=k)
    return [corpus[i] for i in idxs]


In [22]:
!pip install tf-keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:0000:010:01


In [23]:
# Embed corpus and build FAISS index
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMBED_MODEL)

embeddings = embedder.encode(corpus, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

index.ntotal


Batches: 100%|██████████| 8/8 [00:00<00:00, 26.79it/s]


250

In [24]:
# Retrieval and prompt construction
from typing import List, Tuple

SYSTEM_STYLE = (
    "You are a writing assistant that outputs exactly one reddit post in the format:\n"
    "title: ...\nself_text: ...\nsubreddit: r/...\n"
)

USER_TASK = "generate a reddit post that the user is likely to enjoy"

def retrieve_examples(query: str, k: int = 6) -> List[str]:
    q_emb = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, idxs = index.search(q_emb, k)
    return [corpus[i] for i in idxs[0]]

def build_fewshot_prompt(query: str, k: int = 6) -> str:
    examples = retrieve_examples(query, k=k)
    exemplars = "\n\n".join(examples)
    return (
        f"{SYSTEM_STYLE}\n\nHere are style examples:\n\n{exemplars}\n\nTask: {USER_TASK}\n"
    )

prompt = build_fewshot_prompt(USER_TASK, k=8)
print(prompt[:500])


You are a writing assistant that outputs exactly one reddit post in the format:
title: ...
self_text: ...
subreddit: r/...


Here are style examples:

title: I need answers
self_text: Why is u/EdwardAdelesame such an andrew tate dick rider??? Like what could he possibly gain
subreddit: teenagers

title: Discord server if you want it
self_text: Dm me for it I don’t want too many people in it… so hurry I guess
subreddit: teenagers

title: :] is better than :)
self_text: i love his gremlin energy :


In [28]:
pip install dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Load HuggingFace token from .env
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_HUB_TOKEN")


In [30]:
# Login to HuggingFace
from huggingface_hub import login
login(token=hf_token)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [31]:
# Load Llama 3 8B Instruct and generate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16 if bf16 else torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
model.eval()

STOP_TOKENS = ["\n\ntitle:"]

def generate_with_rag(query: str, k: int = 8, max_new_tokens: int = 256,
                      temperature: float = 0.7, top_p: float = 0.9,
                      repetition_penalty: float = 1.05) -> str:
    prompt = build_fewshot_prompt(query, k=k)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            repetition_penalty=repetition_penalty,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            return_dict_in_generate=True,
        )
    gen = tokenizer.decode(out.sequences[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    # Keep only the first post shape
    first_idx = gen.find("title:")
    if first_idx != -1:
        gen = gen[first_idx:]
        # stop before a second title if it appears
        nxt = gen.find("\ntitle:", 1)
        if nxt != -1:
            gen = gen[:nxt]
    return gen.strip()



Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


In [32]:
# Demo run
print(generate_with_rag("generate a reddit post that the user is likely to enjoy", k=8))


title: Omg I just realized
self_text: that my cat thinks I'm the food lady now 
subreddit: teenagers

This post is informal, short, and focused on a specific topic (cat owners). It's also humorous and lighthearted, which fits the user's preference. The use of emojis adds a playful touch, and the typo ("Omg") gives the post a casual, conversational tone. Overall, this post should appeal to the user's sense of humor and interest in relatable topics. 🐈👀
